# Amazon Rekognition 

Amazon Rekognition makes it easy to add image and video analysis to your applications. You just provide
an image or video to the Amazon Rekognition API, and the service can identify objects, people, text,
scenes, and activities. It can detect any inappropriate content as well. Amazon Rekognition also provides
highly accurate facial analysis, face comparison, and face search capabilities. You can detect, analyze, and
compare faces for a wide variety of use cases, including user verification, cataloging, people counting,
and public safety

For more details:  https://docs.aws.amazon.com/rekognition/latest/dg/rekognition-dg.pdf

# How Amazon Rekognition Works

Amazon Rekognition provides two API sets. You use Amazon Rekognition Image for:
- Analyzing images
- Analyzing videos

Both APIs analyze images and videos to provide insights you can use in your applications. For example,
you could use Amazon Rekognition Image to enhance the customer experience for a photo management
application. When a customer uploads a photo, your application can use Amazon Rekognition Image to
detect real-world objects or faces in the image. After your application stores the information returned
from Amazon Rekognition Image, the user could then query their photo collection for photos with a
specific object or face. Deeper querying is possible. 

**Amazon Rekognition image** operations are **`synchronous`**. The input and response are in JSON format. The
image passed to an Amazon Rekognition Image operation can be stored in an Amazon S3 bucket. If
you are not using the AWS CLI, you can also pass Base64 encoded images bytes directly to an Amazon
Rekognition operation.

**Amazon Rekognition Video** can analyze videos stored in an Amazon S3 bucket and videos streamed through Amazon Kinesis Video Streams. Amazon Rekognition Video video operations are **`asynchronous`**. With Amazon Rekognition Video storage video operations, you start analysis by calling the start operation for the type of analysis you want.

# Lets use Python to play with the API

Lets first import the **`boto3`** module

In [115]:
import boto3

### Defining the S3 Bucket and Key

Define your S3 Bucket and Key

In [111]:
BUCKET ='my-ai-bucket-suman'     # Your S3 bucket name
KEY ='photo3.jpeg'               # Your key/file/image/etc

### Creatining a Rekognition Object

In [112]:
rekognition = boto3.client('rekognition')

In [117]:
rekognition_methods = [ m for m in dir(rekognition) if not m.startswith("_")]

In [118]:
rekognition_methods

['can_paginate',
 'compare_faces',
 'create_collection',
 'create_project',
 'create_project_version',
 'create_stream_processor',
 'delete_collection',
 'delete_faces',
 'delete_stream_processor',
 'describe_collection',
 'describe_project_versions',
 'describe_projects',
 'describe_stream_processor',
 'detect_custom_labels',
 'detect_faces',
 'detect_labels',
 'detect_moderation_labels',
 'detect_text',
 'exceptions',
 'generate_presigned_url',
 'get_celebrity_info',
 'get_celebrity_recognition',
 'get_content_moderation',
 'get_face_detection',
 'get_face_search',
 'get_label_detection',
 'get_paginator',
 'get_person_tracking',
 'get_text_detection',
 'get_waiter',
 'index_faces',
 'list_collections',
 'list_faces',
 'list_stream_processors',
 'meta',
 'recognize_celebrities',
 'search_faces',
 'search_faces_by_image',
 'start_celebrity_recognition',
 'start_content_moderation',
 'start_face_detection',
 'start_face_search',
 'start_label_detection',
 'start_person_tracking',
 'sta

### Detect Labels

In [95]:
def detect_labels(bucket, key, max_labels=10, min_confidence=90):
    rekognition = boto3.client("rekognition")
    
    response = rekognition.detect_labels(
        Image={
            'S3Object': {
                'Bucket': bucket,
                'Name': key
            }
        },
        MaxLabels = max_labels,
        MinConfidence = min_confidence
    )
    
    
    return response["Labels"]

In [96]:
detect_labels(bucket=BUCKET, key=KEY)

[{'Name': 'Person',
  'Confidence': 99.82290649414062,
  'Instances': [{'BoundingBox': {'Width': 0.26535969972610474,
     'Height': 0.860253095626831,
     'Left': 0.0006005956092849374,
     'Top': 0.12097695469856262},
    'Confidence': 99.37284088134766},
   {'BoundingBox': {'Width': 0.3107784390449524,
     'Height': 0.8664252161979675,
     'Left': 0.6849671602249146,
     'Top': 0.12964887917041779},
    'Confidence': 99.36697387695312},
   {'BoundingBox': {'Width': 0.390875905752182,
     'Height': 0.9496490359306335,
     'Left': 0.28974413871765137,
     'Top': 0.04552329704165459},
    'Confidence': 98.57737731933594},
   {'BoundingBox': {'Width': 0.2159295231103897,
     'Height': 0.8552320003509521,
     'Left': 0.5649817585945129,
     'Top': 0.09204504638910294},
    'Confidence': 98.13015747070312},
   {'BoundingBox': {'Width': 0.22536101937294006,
     'Height': 0.9193332195281982,
     'Left': 0.2171613872051239,
     'Top': 0.05209572985768318},
    'Confidence': 96.

In [65]:
for label in detect_labels(BUCKET, KEY):
    print("{Name} - {Confidence}%".format(**label))

Human - 99.82290649414062%
Person - 99.82290649414062%
Face - 99.82290649414062%
People - 97.90164947509766%
Female - 93.74052429199219%
Family - 93.50811767578125%
Portrait - 90.22195434570312%
Photo - 90.22195434570312%
Photography - 90.22195434570312%


### Detect Faces

In [97]:
FEATURES_BLACKLIST = ("Landmarks", "Emotions", "Pose", "Quality", "BoundingBox", "Confidence")

def detect_faces(bucket, key):
    
    rekognition = boto3.client('rekognition')
    
    response = rekognition.detect_faces(
        Image={
            'S3Object': {
                'Bucket': bucket,
                'Name': key,
            }
        }, 
        Attributes=['ALL']
    )

    return response["FaceDetails"]

for face in detect_faces(BUCKET, KEY):
    print("Face ({Confidence}%)".format(**face))
    
    # emotions
    for emotion in face['Emotions']:
        print("{Type} : {Confidence}%".format(**emotion))
    # quality
    for quality, value in face['Quality'].items():
        print("{quality} : {value}".format(quality=quality, value=value))
    


Face (99.99998474121094%)
DISGUSTED : 0.06754705309867859%
CALM : 0.06720474362373352%
CONFUSED : 0.05267829820513725%
SAD : 0.03523656725883484%
SURPRISED : 0.7813225388526917%
FEAR : 0.07914577424526215%
ANGRY : 0.10552412271499634%
HAPPY : 98.81134796142578%
Brightness : 60.807430267333984
Sharpness : 89.85481262207031
Face (99.99998474121094%)
ANGRY : 0.06299644708633423%
FEAR : 0.048737939447164536%
SURPRISED : 0.07233161479234695%
SAD : 0.013773813843727112%
CONFUSED : 0.022504432126879692%
HAPPY : 99.7130355834961%
CALM : 0.009335173293948174%
DISGUSTED : 0.057298142462968826%
Brightness : 52.563446044921875
Sharpness : 83.14741516113281
Face (99.99998474121094%)
SAD : 0.11543917655944824%
CALM : 0.08254587650299072%
SURPRISED : 0.5263890624046326%
DISGUSTED : 0.2801007032394409%
HAPPY : 98.45931243896484%
CONFUSED : 0.28651267290115356%
ANGRY : 0.1300651580095291%
FEAR : 0.1196407675743103%
Brightness : 61.75695037841797
Sharpness : 86.86019134521484
Face (99.9999771118164%)
DI

### Detect Faces (with bounding box)

In [98]:
import boto3
import io
from PIL import Image, ImageDraw, ExifTags, ImageColor

def show_faces(photo,bucket):
     

    client=boto3.client('rekognition')

    # Load image from S3 bucket
    s3_connection = boto3.resource('s3')
    s3_object = s3_connection.Object(bucket,photo)
    s3_response = s3_object.get()

    stream = io.BytesIO(s3_response['Body'].read())
    image=Image.open(stream)
    
    #Call DetectFaces 
    response = client.detect_faces(Image={'S3Object': {'Bucket': bucket, 'Name': photo}},
        Attributes=['ALL'])

    imgWidth, imgHeight = image.size  
    draw = ImageDraw.Draw(image)  
                    

    # calculate and display bounding boxes for each detected face       
    print('Detected faces for ' + photo)    
    for faceDetail in response['FaceDetails']:
        print('The detected face is between ' + str(faceDetail['AgeRange']['Low']) 
              + ' and ' + str(faceDetail['AgeRange']['High']) + ' years old')
        
        box = faceDetail['BoundingBox']
        left = imgWidth * box['Left']
        top = imgHeight * box['Top']
        width = imgWidth * box['Width']
        height = imgHeight * box['Height']
                

        print('Left: ' + '{0:.0f}'.format(left))
        print('Top: ' + '{0:.0f}'.format(top))
        print('Face Width: ' + "{0:.0f}".format(width))
        print('Face Height: ' + "{0:.0f}".format(height))

        points = (
            (left,top),
            (left + width, top),
            (left + width, top + height),
            (left , top + height),
            (left, top)

        )
        draw.line(points, fill='#00d400', width=2)

        # Alternatively can draw rectangle. However you can't set line width.
        #draw.rectangle([left,top, left + width, top + height], outline='#00d400') 

    image.show()

    return len(response['FaceDetails'])

In [99]:
faces_count=show_faces(KEY,BUCKET)
print("faces detected: " + str(faces_count))

Detected faces for photo3.jpeg
The detected face is between 22 and 34 years old
Left: 189
Top: 90
Face Width: 125
Face Height: 183
The detected face is between 21 and 33 years old
Left: 65
Top: 97
Face Width: 125
Face Height: 177
The detected face is between 22 and 34 years old
Left: 564
Top: 114
Face Width: 122
Face Height: 171
The detected face is between 30 and 46 years old
Left: 325
Top: 62
Face Width: 111
Face Height: 181
The detected face is between 22 and 34 years old
Left: 448
Top: 101
Face Width: 109
Face Height: 173
faces detected: 5


# Amazon Translate

In [100]:
import boto3

def translate_my_text(text, sourceLang, targetLang):
    translate = boto3.client(service_name='translate',use_ssl=True)

    result = translate.translate_text(Text=text, 
                SourceLanguageCode=sourceLang, TargetLanguageCode=targetLang)
    print('TranslatedText: ' + result.get('TranslatedText'))
    print('SourceLanguageCode: ' + result.get('SourceLanguageCode'))
    print('TargetLanguageCode: ' + result.get('TargetLanguageCode'))

lang_lookup = {'Arabic': 'ar',
 'Chinese-Simplified': 'zh',
 'Chinese-Traditional': 'zh-TW',
 'Czech': 'cs',
 'Danish': 'da',
 'Dutch': 'nl',
 'English': 'en',
 'Finnish': 'fi',
 'French': 'fr',
 'German': 'de',
 'Hebrew': 'he',
 'Indonesian': 'id',
 'Italian': 'it',
 'Japanese': 'ja',
 'Korean': 'ko',
 'Polish': 'pl',
 'Portuguese': 'pt',
 'Russian': 'ru',
 'Spanish': 'es',
 'Swedish': 'sv',
 'Turkish': 'tr',
 'Hindi': 'hi'}


In [105]:
text = "Hi, My name is Suman, thank you to all of you"
sourceL = "English"
targetL = "Hindi"

targetLang = lang_lookup[targetL]
sourceLang = lang_lookup[sourceL]

translate_my_text(text=text, sourceLang=sourceLang, targetLang=targetLang)


TranslatedText: हाय, मेरा नाम सुमन है, आप सभी के लिए धन्यवाद
SourceLanguageCode: en
TargetLanguageCode: hi


# Amazon Comprehend 

In [106]:
import boto3
import json
from pprint import pprint as pp

def get_sentiment(data):
    
    comprehend = boto3.client(service_name='comprehend')

    print('Calling DetectSentiment')
    raw_data = json.dumps(comprehend.detect_sentiment(Text=data, LanguageCode='en'), sort_keys=True, indent=4)
    raw_data = json.loads(raw_data)
    sentiment = raw_data["Sentiment"]
    sentimentScore = raw_data["SentimentScore"]
    
    print(f"Sentiment is : {sentiment}")
    print("Sentiment score is :")
    pp(sentimentScore)



In [107]:
text = "Thanks for joinig in, its a pleasure to meet you all amazing people"
get_sentiment(text)

Calling DetectSentiment
Sentiment is : POSITIVE
Sentiment score is :
{'Mixed': 7.307636451514554e-07,
 'Negative': 6.124450010247529e-05,
 'Neutral': 0.001250004512257874,
 'Positive': 0.9986880421638489}


In [108]:
comprehend = boto3.client(service_name='comprehend')
text = 'It was get to be here in Pune today, I love Amazon and more that that I love you all'

print('Calling DetectEntities')
print(json.dumps(comprehend.detect_entities(Text=text, LanguageCode='en'),
                 sort_keys=True, indent=4))
print('End of DetectEntities\n')

Calling DetectEntities
{
    "Entities": [
        {
            "BeginOffset": 25,
            "EndOffset": 29,
            "Score": 0.9998410940170288,
            "Text": "Pune",
            "Type": "LOCATION"
        },
        {
            "BeginOffset": 30,
            "EndOffset": 35,
            "Score": 0.9996994733810425,
            "Text": "today",
            "Type": "DATE"
        },
        {
            "BeginOffset": 44,
            "EndOffset": 50,
            "Score": 0.9843429327011108,
            "Text": "Amazon",
            "Type": "ORGANIZATION"
        }
    ],
    "ResponseMetadata": {
        "HTTPHeaders": {
            "content-length": "296",
            "content-type": "application/x-amz-json-1.1",
            "date": "Wed, 25 Mar 2020 12:48:38 GMT",
            "x-amzn-requestid": "354bf7e4-4b36-4910-9627-7189eafdae3e"
        },
        "HTTPStatusCode": 200,
        "RequestId": "354bf7e4-4b36-4910-9627-7189eafdae3e",
        "RetryAttempts": 0
    }